In [ ]:
#Loading it up
reviews_train = []
reviews_test = []

for line in open('aclImdb/movie_data/full_train.txt'):
    reviews_train.append(line.strip().lower())
    
for line in open('aclImdb/movie_data/full_test.txt'):
    reviews_test.append(line.strip().lower())
    


In [ ]:
#Cleanup of Data
import re

reviews_train = [re.compile("[.;:!\'?,\"()\[\]]").sub("", line) for line in reviews_train]
reviews_test = [re.compile("[.;:!\'?,\"()\[\]]").sub("", line) for line in reviews_test]

reviews_train = [re.compile("(<br\s*/><br\s*/>)|(-)|(/)").sub(" ", line) for line in reviews_train]
reviews_test = [re.compile("(<br\s*/><br\s*/>)|(-)|(/)").sub(" ", line) for line in reviews_test]

print(reviews_test[:2])


In [ ]:
# One hot encode
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train)
X = cv.transform(reviews_train)
X_test = cv.transform(reviews_test)

In [ ]:
# LOGISTIC REGRESSION - Finding optimal value for c
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

target = [1 if i<12500 else 0 for i in range(25000)]

X_train, X_val, Y_train, Y_val = train_test_split(X, target, train_size=0.75)

for c in [0.01, 0.05, 0.1, 0.5, 1]:
    lr = LogisticRegression(C = c)
    lr.fit(X_train, Y_train)

    print("Accuracy for C=%s is %s" %(c, accuracy_score(Y_val, lr.predict(X_val))))



In [ ]:
# Moving on with c = 0.05
lr = LogisticRegression(C = 0.05)
lr.fit(X, target)

print("Final Accuracy is %s" %(accuracy_score(target, lr.predict(X_test))))

In [ ]:
# Sanity Check

feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), lr.coef_[0]
    )
}

for worst in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]: print(worst)
print("\n")
for best in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]: print(best)
    

In [ ]:
#Test input
import numpy as np

lr.predict(cv.transform(['An entertaining and engaging look at the world of Indian rappers.']))



In [101]:
import pickle

pickle.dump(lr, open('logRegSentiment.pkl', 'wb'))